<a href="https://colab.research.google.com/github/lerrwee/yandex_images_analysis/blob/main/yandex_images_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Задача по анализу поиска в Яндекс Картинках**

In [ ]:
df = pd.read_csv('data.tsv', sep='\t', header=None, names=['query', 'timestamp', 'platform'])

df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df['date'] = df['datetime'].dt.date
df['hour'] = df['datetime'].dt.hour

Первым шагом необходимо определить временной период, за который предоставлены данные. Для этого выполняется SQL-запрос, который находит минимальную и максимальную даты в датасете.
Анализ проводился на основе данных о поисковых запросах в Яндекс.Картинках за период **с 31 августа по 21 сентября 2021 года**

In [ ]:
date_range = sqldf("""
    SELECT MIN(date) as start_date, MAX(date) as end_date
    FROM df
""")

print(f"Данные охватывают период с {date_range['start_date'][0]} по {date_range['end_date'][0]}")

Данные охватывают период с 2021-08-31 по 2021-09-21


"Запросы, содержащие слово «ютуб», встречаются чаще на компьютерах **(desktop) — 802** запроса против **597 на мобильных устройствах (touch).** Это может быть связано с тем, что пользователи ПК чаще ищут дополнительные материалы (например, скриншоты, обложки видео), в то время как мобильные пользователи чаще используют само приложение YouTube.

In [ ]:
youtube_queries = sqldf("""
    SELECT platform, COUNT(*) as count
    FROM df
    WHERE query LIKE '%ютуб%'
    GROUP BY platform
""")

fig = px.bar(youtube_queries, x='platform', y='count',
             title='Запросы с "ютуб" по платформам',
             color='platform', text='count')
fig.update_traces(textposition='outside')
fig.show()

Топ-10 запросов значительно различается в зависимости от платформы:
*   На desktop преобладают учебные и рабочие запросы: «календарь 2021», «таблица Менделеева», «английский алфавит».
*   На touch лидируют развлекательные и бытовые запросы: «порно», «секс», «с днём рождения».

Это подтверждает гипотезу о том, что пользователи компьютеров чаще используют поиск для работы и учебы, а мобильные пользователи — для развлечения и повседневных нужд.

In [ ]:
top_desktop = sqldf("""
    SELECT query, COUNT(*) as count
    FROM df
    WHERE platform = 'desktop'
    GROUP BY query
    ORDER BY count DESC
    LIMIT 10
""")

top_touch = sqldf("""
    SELECT query, COUNT(*) as count
    FROM df
    WHERE platform = 'touch'
    GROUP BY query
    ORDER BY count DESC
    LIMIT 10
""")

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Топ-10 Desktop', 'Топ-10 Touch'),
                    horizontal_spacing=0.15)

fig.add_trace(
    go.Bar(
        y=top_desktop['query'],
        x=top_desktop['count'],
        orientation='h',
        name='Desktop',
        marker_color='#1f77b4',
        text=top_desktop['count'],
        textposition='auto'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        y=top_touch['query'],
        x=top_touch['count'],
        orientation='h',
        name='Touch',
        marker_color='#ff7f0e',
        text=top_touch['count'],
        textposition='auto'
    ),
    row=1, col=2
)

fig.update_layout(
    height=600,
    showlegend=False,
    yaxis=dict(autorange="reversed"),
    yaxis2=dict(autorange="reversed"),
    hovermode='y unified',
    title_text="Сравнение топ-10 запросов по платформам",
    margin=dict(l=100, r=100, t=80, b=80)
)

fig.update_xaxes(title_text="Количество запросов", row=1, col=1)
fig.update_xaxes(title_text="Количество запросов", row=1, col=2)
fig.update_yaxes(title_text="Запросы", row=1, col=1)
fig.update_yaxes(title_text="Запросы", row=1, col=2)

fig.show()

Активность поиска существенно зависит от времени суток:
*   Пик на desktop приходится на утро **(9:00–10:00)** — вероятно, пользователи ищут информацию для работы или учебы.
*   Пик на touch наблюдается вечером **(17:00–18:00)** — это может быть связано с использованием смартфонов в свободное время.

Такое различие подтверждает, что компьютеры чаще используются днём, а мобильные устройства — вечером.

In [ ]:
hourly_activity = sqldf("""
    SELECT hour, platform, COUNT(*) as count
    FROM df
    GROUP BY hour, platform
    ORDER BY hour
""")

fig = px.line(hourly_activity, x='hour', y='count', color='platform',
              title='Активность поиска по часам',
              markers=True)
fig.update_xaxes(title='Час дня', range=[0, 23], dtick=1)
fig.update_yaxes(title='Количество запросов')
fig.show()

Категоризация запросов показала, что:
*   На desktop доминируют нейтральные темы (94%), такие как поиск изображений для работы.
*   На touch значительно выше доля взрослого контента (6,3% против 1,5% на desktop) и поздравлений (1,5% против 0,2%).

Это говорит о том, что мобильные пользователи чаще ищут развлекательный и персонифицированный контент.

In [ ]:
def categorize_query(query):
    query = query.lower()
    if 'порно' in query or 'секс' in query or 'porno' in query:
        return 'adult'
    elif 'день рождения' in query or 'доброе утро' in query:
        return 'greetings'
    elif 'айфон' in query or 'iphone' in query:
        return 'technology'
    elif 'фото' in query or 'картинки' in query:
        return 'images'
    elif 'собака' in query or 'кошка' in query:
        return 'pets'
    else:
        return 'other'

df['category'] = df['query'].apply(categorize_query)

category_colors = {
    'adult': '#FF6B6B',
    'greetings': '#4ECDC4',
    'technology': '#45B7D1',
    'images': '#FFBE0B',
    'pets': '#A05195',
    'other': '#D3D3D3'
}

desktop_data = df[df['platform'] == 'desktop']['category'].value_counts().reset_index()
desktop_data.columns = ['category', 'count']
touch_data = df[df['platform'] == 'touch']['category'].value_counts().reset_index()
touch_data.columns = ['category', 'count']

fig = make_subplots(rows=1, cols=2,
                   specs=[[{'type':'domain'}, {'type':'domain'}]],
                   subplot_titles=("Desktop", "Touch"))

fig.add_trace(go.Pie(
    labels=desktop_data['category'],
    values=desktop_data['count'],
    name="Desktop",
    marker_colors=[category_colors[c] for c in desktop_data['category']],
    hole=0.3,
    textinfo='percent+label',
    hoverinfo='label+percent+value',
    textposition='inside'
), 1, 1)

fig.add_trace(go.Pie(
    labels=touch_data['category'],
    values=touch_data['count'],
    name="Touch",
    marker_colors=[category_colors[c] for c in touch_data['category']],
    hole=0.3,
    textinfo='percent+label',
    hoverinfo='label+percent+value',
    textposition='inside'
), 1, 2)

fig.update_layout(
    title_text="Распределение категорий запросов по платформам",
    height=500,
    annotations=[
        dict(text='Desktop', x=0.18, y=0.5, font_size=20, showarrow=False),
        dict(text='Touch', x=0.82, y=0.5, font_size=20, showarrow=False)
    ]
)
fig.show()